# Planar Straight-Line Graph & Doubly-Connected Edge List

## Введение

#### Откуда возникает задача

Для того, чтобы лучше понимать, о чём пойдёт речь, представим себе некоторую карту местности.Она может быть совершенно любой. На ней могут быть отмечены реки, железнодорожные маршруты, точки с названиями населённых пунктов и так далее. 

Пусть мы захотели найти на карте некоторый населённый пункт, тогда нанесённые на карту названия рек будут нас только отвлекать. Поэтому, чтобы такой проблемы не возникало, в географических системах карты делят на тематические слои. Например, один слой содержит только дороги, другой слой только реки и так далее. Каждый слой содержит только один тип информации, при этом по смыслу эта информация может быть совершенно любой.

С геометрической точки зрения, слои могут быть совершенно разными. Например, слой с дорогами представляет собой множество отрезков, а слой с городами будет простым множеством точек. А если разбить некоторый город на районы, то они будут представлять собой разбиение карты на области. 

Люди, которые пользуются такой системой карт со слоями, могут довольно легко найти то, что им нужно. Допустим, человеку нужно проложить маршрут между двумя городами. Тогда сначала он открывает слой с городами, находит нужные ему, а потом открывает слой с дорогами и ищет маршрут между ними. 

Современные системы такого типа позволяют ещё и брать пересечения нескольких слоёв. Допустим, что у нас есть слой дорог и слой рек. Тогда пересечение этих слоёв будет представлять собой множество мостов. 

#### Постановка задачи

Слои на рассматриваемой карте могут быть совершенно любыми. Как самый тривиальный пример, можно взять два слоя, каждый из которых является набором отрезков. Тогда пересечение таких слоев сводится к задаче пересечения отрезков.

Более общая ситуация, рассматриваемая в данном конспекте, когда слои представляют собой разбиение плоскости на некоторые области. И прежде чем работать с такими областями, нужно разработать некоторую удобную для этого структуру. Хранить области в виде набора отрезков — не очень хорошая идея, так как некоторые операции (например, запрос границы области) будут выполняться достаточно долго. 

Намного удобнее хранить некоторую структурную, топологическую информацию: какие отрезки принадлежат данной области, какие области смежны друг с другом и так далее.

Итого, задача которую мы хотим решить — это построение удобной структуры для работы с некоторыми областями на плоскости. 

## Определения

**ППЛГ** (англ. _PSLG_) — Плоский (планарный) прямолинейный граф. Граф называется планарным, если его можно нарисовть на плоскости так, чтобы его ребра не пересекались. Планарный граф называется прямолинейным, если каждое его ребро — это прямая линия.

**Область** (англ. _face_) — пусть дан ППЛГ. Тогда _область_ — это множество точек, которое ограничено по периметру рёбрами графа.

**Разбиение плоскости** — некоторе разделение плоскости на области. Заметим, что любой плоский граф разбивает плоскость на несколько областей (граней). Одна из граней не ограничена, ее называют внешней (англ. _external_) гранью, а остальные — внутренними (англ. _unbounded_) гранями.

**РСДС** (англ. _DCEL_) — реберный список с двойными связями. Реберный список с двойными связями особенно удобен для представления ППЛГ.

<center><img src="pictures/planar_graph.png" width="300"></center>

<div style="padding:10px">
_Пример планарного графа. Синим контуром обозначены грани, за исключением внешней грани (всего 5 граней). Обратите внимание, что внутри грани могут содержаться другие ребра и вершины._
</div>


### Описание РСДС

#### Формальное: 
Пусть задан граф _G = (V, E)_, _V = {$v_1$, $v_2$, ... $v_n$}_, _E = {$e_1$, $e_2$, ... $e_m$}_. Главная компонента РСДС для планарного графа это реберный узел. Между ребрами графа и реберными узлами РСДС существует биективное отображение, т.е. каждое ребро однозначно представлено в РСДС ровно один раз. Реберный узел РСДС, соответствующий ребру графа, например, $e_k$ = {$v_i$, $v_j$} имеет 4 поля ($v_1$, $v_2$, $f_1$, $f_2$) и 2 указателя ($p_1$, $p_2$). Поле $v_1$ содержит начало ребра, а поле $v_2$ содержит его конец (так изначально неориентированное ребро получает условную ориентацию). Поля $f_1$ и $f_2$ содержат имена граней, лежащих слева и справа от ориентированного ребра ($v_1$, $v_2$). Указатель $p_1$ (и, соответственно, $p_2$) задает реберный узел, содержащий первое ребро, встречаемое вслед за ребром ($v_1$, $v_2$), при повороте от него против часовой стрелки вокруг $v_1$ (и, соответственно, $v_2$).

#### Нефромальное:

РСДС состоит из трех компонент:
1. _Vertex_ — вершина. Содержит координаты точки, а также указатель на инцидентное ребро.
2. _Face_ — область. Содержит указатель на некоторое ребро на его границе. Для неограниченных поверхностей это nil. Также содержит список указателей на внутренние компоненты (дырки), то есть, по указателю на одно из инцидентных каждой дырке рёбер (nil, если дырок нет).
3. _HalfEdge_ — ребро (полуребро). Содержит указатель на вершину, откуда исходит (origin), указатель на ребро близнец, направленное в другую сторону (twin), инцидентную поверхность (incident_face), и указатели на следующее и предыдущее ребра.

<center><img src="pictures/dcel.png" width="700"></center>



## Применение 

Итак, мы сказали, что разбиение плоскости может быть порождено планарным прямолинейным графом. То есть, сам граф, по сути, уже является структурой, которая может представлять слой в карте. Но от такой структуры пользы будет не очень много. РСДС, в свою очередь, является более удобным представлением, которое позволяет решать различные задачи.

### Пример: получить смежные области для ребра
Для этого достаточно взять инцидетную область для данного ребра и для его близнеца

### Пример: перебрать все ребра инцидентные данной вершине по часовой стрелке
Для этого нужно взять ребро, инцидентное данной вершине, из него перейти в twin, потом в next. Повторять, пока не придем в первое ребро.

# Построение РСДС множества прямых

План алгоритма, взято с викиконспектов, будет переписано.

Пусть у нас есть множество прямых. Хотим построить РСДС по этому множеству прямых.

Изначально у нас есть _face_, представляющий все плоскость. Будем добавлять прямые по одной следующим образом:

1. Локализовать некотоую точку данной прямой в _face_.
2. Найти _HalfEdge_'и, пересекающие данную прямую.
3. Разбить текущий _face_ на два: _face1_ и _face2_.
* Разбиваем _HalfEdge_'и, пересекающие добавляюемую прямую на 2 новых _HalfEdge_. 
* Создаем новый _HalfEdge_, соответствующий отрезку добавляемой прямой, попадающий в данный _face_.
* Обновляем все указатели в нужных _HalfEdge_.
* У _HalfEdge_ исходного _face_ обновляем поле _incident_face_ на _face1_ и _face2_ соответственно.
4. Мы знаем в какие _face_ пошла наша прямая. Запускаемся в этих _face_. Если несколько _HalfEdge_ пересекались в одной точке на данной прямой, то необходимо перебрать все и перейти в нужный _face_.